In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting_for_tcga as setting_0

import setting_for_ccle as setting_1

title = '{}<br>on<br>{}'.format(
    setting_1.NAME,
    setting_0.NAME,
)

print(title)

path_dict_0 = path(setting_0)

path_dict_1 = path(setting_1)

In [ ]:
feature_x_sample_0 = pd.read_table(
    path_dict_0['feature_x_sample_file_path'],
    index_col=0,
)

gps_map_0 = ccal.load_gps_map(path_dict_0['gps_map_file_path'])

w_0 = pd.DataFrame(
    gps_map_0.wt.T,
    index=gps_map_0.wt_elements,
    columns=gps_map_0.nodes,
)

h_0 = pd.DataFrame(
    gps_map_0.h,
    index=gps_map_0.nodes,
    columns=gps_map_0.h_elements,
)

In [ ]:
feature_x_sample_1 = ccal.read_and_process_feature_x_sample(
    setting_1.FEATURE_X_SAMPLE_FILE_PATH,
    nanize=setting_0.NANIZE,
    drop_na_axis=setting_0.DROP_NA_AXIS,
    min_n_not_na_unique_value=setting_0.MIN_N_NOT_NA_UNIQUE_VALUE,
    max_na=setting_0.MAX_NA,
    shift_as_necessary_to_achieve_min_before_logging=setting_0.SHIFT_AS_NECESSARY_BEFORE_LOGGING,
    log_base=setting_0.LOG_BASE,
    normalization_axis=setting_0.NORMALIZATION_AXIS,
    normalization_method=setting_0.NORMALIZATION_METHOD,
    max_plot_n=setting_0.MAX_PLOT_N,
    plot=False,
)

feature_x_sample_1 = feature_x_sample_1.loc[w_0.index.str.lstrip('(-+) ').unique()]

In [ ]:
signal_matrix_1 = np.full(
    (
        w_0.shape[0],
        feature_x_sample_1.shape[1],
    ),
    np.nan,
)

feature_skew_t_pdf_fit_parameter_0 = pd.read_table(
    path_dict_0['feature_skew_t_pdf_fit_parameter_file_path'],
    index_col=0,
)

n = w_0.shape[0]

n_per_print =  n // 10

for i, sign_feature in enumerate(w_0.index):
    
    if i % n_per_print == 0:
        
        print('{}/{} ...'.format(
            i,
            n,
        ))
    
    sign, feature = sign_feature.split()
    
    location_0, scale_0, degree_of_freedom_0, shape_0 = feature_skew_t_pdf_fit_parameter_0.loc[feature, [
        'Location',
        'Scale',
        'Degree of Freedom',
        'Shape',
    ]]

    context_dict_0 = ccal.compute_context(
        feature_x_sample_0.loc[feature].values,
        location=location_0,
        scale=scale_0,
        degree_of_freedom=degree_of_freedom_0,
        shape=shape_0,
        scale_with_kl=setting_0.SCALE_WITH_KL,
    )
    
    grid_0 = context_dict_0['grid']

    context_indices_0 = context_dict_0['context_indices']
    
    if sign == '(-)':

        context_indices_0 = -context_indices_0.clip(max=0)
    
    elif sign == '(+)':
        
        context_indices_0 = context_indices_0.clip(min=0)
        
#     training__context_indices = ccal.normalize_nd_array(
#         context_indices_0,
#         None,
#         '0-1',
#     )

    feature_values_1 = feature_x_sample_1.loc[feature].values

    not_nan_1 = ~np.isnan(feature_values_1)

    signal_matrix_1[i][not_nan_1] = context_indices_0[[np.absolute(grid_0 - value).argmin() for value in feature_values_1[not_nan_1]]]

signal_matrix_1 = pd.DataFrame(
    signal_matrix_1, 
    index=w_0.index,
    columns=feature_x_sample_1.columns,
)

for sign_feature in np.random.choice(
    signal_matrix_1.index,
    size=16,
    replace=False,
):

    sign, feature = sign_feature.split()
    
    feature_values_1 = feature_x_sample_1.loc[feature].sort_values()
    
    feature_signals_1 = signal_matrix_1.loc[sign_feature, feature_values_1.index]

    ccal.plot_context(
        feature_x_sample_0.loc[feature],
        scale_with_kl=setting_0.SCALE_WITH_KL,
        title=sign_feature,
    )

    ccal.plot_points(
        ('Look Up Context', ),
        (feature_values_1, ),
        (feature_signals_1, ),
        texts=(feature_values_1.index, ),
    )

In [ ]:
pull_power = setting_0.GPS_MAP_H_PULL_POWER

element_marker_size = setting_0.GPS_MAP_H_ELEMENT_MARKER_SIZE

In [ ]:
a_1 = signal_matrix_1.fillna(0)

h_1 = ccal.solve_for_nmf_h(
    a_1,
    w_0,
#     pd.DataFrame(
#         ccal.normalize_nd_array(
#             w_0.values,
#             0,
#             'sum',
#         ),
#         index=w_0.index,
#         columns=w_0.columns,
#     ),
    method='nnls',
)

h_element_states_1 = gps_map_0.predict(
    'h',
    h_1,
    pull_power=pull_power,
    title=title,
    element_marker_size=element_marker_size,
)

In [ ]:
ccal.plot_heat_map(
    h_1,
    normalization_axis=0,
    normalization_method='-0-',
    column_annotation=h_element_states_1,
    cluster_axis=1,
    title=title,
    xaxis_title=h_1.columns.name,
    yaxis_title=h_1.index.name,
)


ccal.make_comparison_panel(
    h_1,
    h_1,
    name_0='Predicted',
    name_1='Predicted',
)

known = pd.read_table(
        path_dict_1['h_file_path'],
        index_col=0,
    )

ccal.make_comparison_panel(
    h_1,
    known,
    name_0='Predicted',
    name_1='Known',
)

In [ ]:
feature_dicts_1 = setting_1.make_feature_dicts()

In [ ]:
feature_dicts_1_ = {feature_group: feature_dict for feature_group, feature_dict in feature_dicts_1.items() if feature_group in (
    'NP24 Compound',
    'CTRP Compound',
    'Achilles RNAi',
    'Achilles CRISPR',
)}

In [ ]:
state_x_sample = ccal.make_membership_df_from_categorical_series(pd.read_table(
    path_dict_1['h_hcc__k_x_column_file_path'],
    index_col=0,
).loc['K{}'.format(setting_1.H_HCC_K)])

state_x_sample.index = ('S{}'.format(i) for i in state_x_sample.index)

feature_dicts_1_['State (original)'] = {
    'df': state_x_sample,
    'data_type': 'binary',
    'emphasis':'high',
}

In [ ]:
targets = ccal.make_membership_df_from_categorical_series(h_element_states_1)

targets.index = ('S{}'.format(i) for i in targets.index)

n_target = targets.shape[0]

ccal.make_match_panels(
    (targets.iloc[i] for i in range(n_target)),
    (
        False,
    ) * n_target,
    (
        False,
    ) * n_target,
    (
        'binary',
    ) * n_target,
    feature_dicts_1_,
    n_job=setting_0.MAX_N_JOB,
    n_required_for_match_function=targets.shape[1] * 0.5,
    extreme_feature_threshold=setting_0.EXTREME_FEATURE_THRESHOLD,
)

In [ ]:
features_to_highlight_1 = make_features_to_highlight()

features_to_highlight_1

In [ ]:
for feature_group, features in features_to_highlight_1.items():
    
    if feature_group in feature_dicts_1:
        
        df = feature_dicts_1[feature_group]['df']
        
        annotation_type = feature_dicts_1[feature_group]['data_type']

        for feature in features:
            
            if feature in df.index:

                h_1_element_states = gps_map_0.predict(
                    'h',
                    h_1,
                    pull_power=pull_power,
                    annotation_x_element=df.loc[feature].to_frame().T,
                    annotation_std_maxs=(3, ),
                    annotation_types=(annotation_type, ),
                    title='{}<br>{}'.format(
                        feature_group,
                        feature,
                    ),
                    element_marker_size=element_marker_size,
                )